In [1]:
%run QM9GNN2_Multitask.ipynb

Loading QM9 dataset.
Reading SDF


In [2]:
import itertools

In [3]:
optimizer = Adam(lr=learning_rate)
loss = 'mse'

In [4]:
def generate_single_task_filename(task):
    return path.join('single_task_trained_models', task + '.h5')

In [19]:
for i in range(0, 2):
    print('learning', tasks[i])
    model = create_single_task_model(X_in, A_in, E_in, n_out)
    model.compile(optimizer=optimizer, loss=loss)
    es_callback = EarlyStopping(monitor='val_loss', patience=es_patience)
    model.fit([X_train, A_train, E_train],
             y_train_list[i],
             batch_size=batch_size,
             validation_split=0.1,
             epochs=epochs,
             callbacks=[es_callback])
    model.save_weights(generate_single_task_filename(tasks[i]))

learning A
Train on 8100 samples, validate on 900 samples
Epoch 1/25
8100/8100 [==============================] - 140s 17ms/sample - loss: 0.9081 - val_loss: 0.9522
Epoch 2/25
8100/8100 [==============================] - 139s 17ms/sample - loss: 0.7268 - val_loss: 0.8841
Epoch 3/25
8100/8100 [==============================] - 138s 17ms/sample - loss: 0.7056 - val_loss: 0.8596
Epoch 4/25
8100/8100 [==============================] - 139s 17ms/sample - loss: 0.6971 - val_loss: 0.8593
Epoch 5/25
8100/8100 [==============================] - 139s 17ms/sample - loss: 0.6915 - val_loss: 0.9159
Epoch 6/25
8100/8100 [==============================] - 136s 17ms/sample - loss: 0.6886 - val_loss: 0.8231
Epoch 7/25
8100/8100 [==============================] - 136s 17ms/sample - loss: 0.6826 - val_loss: 0.8687
Epoch 8/25
8100/8100 [==============================] - 137s 17ms/sample - loss: 0.6741 - val_loss: 0.8454
Epoch 9/25
8100/8100 [==============================] - 138s 17ms/sample - loss: 0.677

In [20]:
def load_and_compile_model(task):
    model = create_single_task_model(X_in=X_in, A_in=A_in, E_in=E_in, n_out=n_out)
    model.load_weights(generate_single_task_filename(task))
    model.compile(optimizer=optimizer, loss=loss)

In [21]:
def calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_transferred_test):
    model_learned = create_single_task_model(X_in=X_in, A_in=A_in, E_in=E_in, n_out=n_out)
    model_transferred = create_single_task_model(X_in=X_in, A_in=A_in, E_in=E_in, n_out=n_out)
    model_learned.load_weights(generate_single_task_filename(task_learned))
    model_transferred.load_weights(generate_single_task_filename(task_transferred))
    
    layers_learned = model_learned.get_weights()
    layers_transferred = model_transferred.get_weights()    
    
    # 10 is a hard-coded value dependent on the architecture
    layers_transferred = layers_learned[:10] + layers_transferred[10:]
    model_transferred.set_weights(layers_transferred)
    model_transferred.compile(optimizer=optimizer, loss=loss)
    
    eval_results = model_transferred.evaluate([X_test, A_test, E_test],
                                              y_transferred_test,
                                              batch_size=batch_size)
    return eval_results

In [22]:
def load_and_test_single_task_model(task_learned, y_test):
    model = create_single_task_model(X_in, A_in, E_in, n_out)
    model.load_weights(path.join('single_task_trained_models', task_learned + '.h5'))
    model.compile(optimizer=optimizer, loss=loss)
    eval_results = model.evaluate([X_test, A_test, E_test], y_test, batch_size=batch_size)
    return eval_results

In [23]:
print(load_and_test_single_task_model('A', y_test_list[0]))
print(calculate_transfer_coefficient_from_file('A', 'B', y_test_list[1]))

1000/1000 [==============================] - 7s 7ms/sample - loss: 0.6330
0.6329849510192871
1000/1000 [==============================] - 7s 7ms/sample - loss: 1.0046
1.0045776519775391


In [10]:
for task, test_data in zip(tasks, y_test_list):
    print(task, load_and_test_single_task_model(task, test_data))

ValueError: Layer #0 (named "edge_conditioned_conv_2" in the current model) was found to correspond to layer edge_conditioned_conv_1 in the save file. However the new layer edge_conditioned_conv_2 expects 4 weights, but the saved weights have 3 elements.

In [11]:
for cluster in clusters:
    losses = load_and_evaluate_multitask_model(cluster, y_test_list)
    for task, loss in zip(cluster, losses):
        print(task, loss)

ValueError: Layer #0 (named "edge_conditioned_conv_4" in the current model) was found to correspond to layer edge_conditioned_conv_1 in the save file. However the new layer edge_conditioned_conv_4 expects 4 weights, but the saved weights have 3 elements.

In [12]:
print(tasks)
transfer_coefficient_dict = dict()
for i, j in itertools.permutations(range(len(tasks)), 2):
    task_learned = tasks[i]
    task_transferred = tasks[j]
    print(task_learned, task_transferred)
    transfer_coefficient = calculate_transfer_coefficient_from_file(task_learned, task_transferred, y_test_list[j])
    print(transfer_coefficient)
    
    if task_learned not in transfer_coefficient_dict.keys():
        transfer_coefficient_dict[task_learned] = {task_transferred: transfer_coefficient}
    else:
        transfer_coefficient_dict[task_learned][task_transferred] = transfer_coefficient

['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv', 'u0_atom', 'u298_atom', 'h298_atom', 'g298_atom']
A B


ValueError: Layer #0 (named "edge_conditioned_conv_6" in the current model) was found to correspond to layer edge_conditioned_conv_1 in the save file. However the new layer edge_conditioned_conv_6 expects 4 weights, but the saved weights have 3 elements.

In [13]:
file = open('cross_task_transfer_coefficients.txt', 'w')
for key1, value in transfer_coefficient_dict.items():
    for key2, coef in value.items():
        print(key1, key2, coef)
        print(key1, key2, coef, file=file)

In [14]:
# The rest of this file is comprised of mostly useless analysis code.

In [15]:
def analyze_transfer_coefficients():
    with open('cross_task_transfer_coefficients.txt', 'r') as file:
        lines = file.readlines()
        transfer_coefficient_dict = dict()
        for line in lines:
            task_learned, task_transferred, transfer_coefficient = line.strip().split()
            if task_learned not in transfer_coefficient_dict.keys():
                transfer_coefficient_dict[task_learned] = {task_transferred: float(transfer_coefficient)}
            else:
                transfer_coefficient_dict[task_learned][task_transferred] = float(transfer_coefficient)
        return transfer_coefficient_dict

In [16]:
transfer_coefficient = analyze_transfer_coefficients()
coef_list = list()
# relevant_tasks = ['alpha', 'homo', 'lumo', 'gap']
relevant_tasks = ['mu', 'homo']
for task_set in itertools.combinations(relevant_tasks, 3):
    coef_sum = 0
    for i, j in itertools.combinations(task_set, 2):
        coef_sum += transfer_coefficient[i][j]**2 + transfer_coefficient[j][i]**2
    coef_list.append((coef_sum, task_set))
# coef_list = np.asarray(coef_list)
# coef_list = np.sort(coef_list)
coef_list.sort()
for entry in coef_list:
    print(entry)

In [17]:
for i, j in itertools.combinations(transfer_coefficient.keys(), 2):
    if transfer_coefficient[i][j] < 0.9 and transfer_coefficient[j][i] < 0.9:
        print(i, 
              j, 
              round(transfer_coefficient[i][j], 3), 
              round(transfer_coefficient[j][i], 3))